# Tutorial: Ontology Interface

<figure style="display: table; text-align:center; margin-left: auto; margin-right:auto">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/simphony/docs/master?filepath=docs%2Fsource%2Fjupyter%2Fwrapper_development.ipynb "Click to run the tutorial yourself!")
    
</figure>


This tutorial introduces the interface to the installed ontologies. The code presented is based on [this example](#). If you want to create your own ontology please refer to [this guide](../yaml.md).

## Background

In the previous tutorial we have discussed the CUDS API. CUDS objects correspond to ontology individuals. In this tutorial we present the API of all other entities: ontology classes, relationships and attributes. These are defined in an ontology file in [YAML](../yaml.md) or [OWL](../owl.md) format. With the presented API you can access the entities and navigate within an ontology.

## Preliminaries

In [1]:
# Install the ontology
!pico install city

INFO 2020-12-02 13:53:47,242 [osp.core.ontology.installation]: Will install the following namespaces: ['city']
INFO 2020-12-02 13:53:47,288 [osp.core.ontology.yml.yml_parser]: Parsing YAML ontology file /mnt/c/Users/dea/Documents/Projects/simphony/osp-core/osp/core/ontology/docs/city.ontology.yml
INFO 2020-12-02 13:53:47,357 [osp.core.ontology.yml.yml_parser]: You can now use `from osp.core.namespaces import city`.
INFO 2020-12-02 13:53:47,358 [osp.core.ontology.parser]: Loaded 403 ontology triples in total
INFO 2020-12-02 13:53:47,397 [osp.core.ontology.installation]: Installation successful


## Accessing entities

First, we want to acces the entities in the ontology. For that, we need to know the installed ontology namespaces. The namespace is stated at the top of the ontology YAML files:

```yaml
---
version: "0.0.3"

namespace: "city"

ontology:
   ...
```

Alternatively you can use pico to see the installed namespaces:

In [2]:
!pico list

Packages:
	- simlammps_ontology
	- city
Namespaces:
	- xml
	- rdf
	- rdfs
	- xsd
	- cuba
	- ns1
	- owl
	- simlammps_ontology
	- city


Knowing the name of the namespace, we can import it in python. With the namespace python object we can access the entities within the namespace:

In [3]:
from osp.core.namespaces import city

print("\nYou can use the namespace to access its entities")
print(city.LivingBeing)

print("\nYou can also use index notation. "
      "For owl ontologies, this queries the entities by rdf.label."
      "It returns a list of all entities with the same label")
print(city.LivingBeing == city["LivingBeing"][0])

print("\nYou can access the namespace of an entity")
print(city is city.LivingBeing.namespace)



You can use the namespace to access its entities
city.LivingBeing

You can also use index notation. For owl ontologies, this queries the entities by rdf.label.It returns a list of all entities with the same label
True

You can access the namespace of an entity
False


## Accessing super- and subclasses

Using the properties `superclasses` and `subclasses` it is easy to navigate the ontology. Direct superclasses and subclasses can also be accessed:

In [4]:
print("\nYou can access the superclasses and the subclasses")
print(city.LivingBeing.superclasses)
print(city.LivingBeing.subclasses)

print("\nYou can access the direct superclasses and subclasses")
print(city.LivingBeing.direct_superclasses)
print(city.LivingBeing.direct_subclasses)

print("\nYou can access a description of the entities")
print(city.LivingBeing.description)

print("\nYou can test if one entity is a subclass / superclass of another")
print(city.Person.is_subclass_of(city.LivingBeing))
print(city.LivingBeing.is_superclass_of(city.Person))


You can access the superclasses and the subclasses
{<OntologyClass cuba.Entity>, <OntologyClass city.LivingBeing>}
{<OntologyClass city.Citizen>, <OntologyClass city.Person>, <OntologyClass city.LivingBeing>}

You can access the direct superclasses and subclasses
{<OntologyClass cuba.Entity>}
{<OntologyClass city.Person>}

You can access a description of the entities
A being that lives

You can test if one entity is a subclass / superclass of another
True
True


## Access entities using a string

Sometimes you only have a string refering to an entity. Using the `get_entity` function you can get the corresponding python object easily:

In [5]:
from osp.core.namespaces import get_entity  # noqa: E402

print("\nYou can get an entity with a string")
print(get_entity("city.LivingBeing"))
print(get_entity("city.LivingBeing") == city.LivingBeing)


You can get an entity with a string
city.LivingBeing
True


## Testing the type of the entities

In the OSP-core YAML ontology files three types of entities can be defined: classes, relationships and attributes. Relationships are subclasses of CUBA.RELATIONSHIP and attributes are subclasses of CUBA.ATTRIBUTE. There are different Python objects for the different entity types. You can use both to check which type of entity you are dealing with:

In [6]:
from osp.core.namespaces import cuba  # noqa: E402

# These are the classes for the ontology entities
from osp.core.ontology import (  # noqa: F401, E402
    OntologyEntity,
    OntologyClass,
    OntologyRelationship,
    OntologyAttribute
)

print("\nYou can test if an entity is a class")
print(isinstance(city.LivingBeing, OntologyClass))
print(not city.LivingBeing.is_subclass_of(cuba.relationship)
      and not city.LivingBeing.is_subclass_of(cuba.attribute))

print("\nYou can test if an entity is a relationship")
print(isinstance(city.hasInhabitant, OntologyRelationship))
print(city.hasInhabitant.is_subclass_of(cuba.relationship))

print("\nYou can test if an entity is a attribute")
print(isinstance(city.name, OntologyAttribute))
print(city.name.is_subclass_of(cuba.attribute))




You can test if an entity is a class
True
True

You can test if an entity is a relationship
True
True

You can test if an entity is a attribute
True
True


## Operations specific to ontology classes

The different types of entities differ in the operations they offer. For classes, you can access the attributes:

In [7]:
print("\nYou can get the attributes of an ontology class and their defaults")
print(city.Citizen.attributes)

print("\nYou can get the non-inherited attributes and their defaults")
print(city.Citizen.own_attributes)
print(city.LivingBeing.own_attributes)


You can get the attributes of an ontology class and their defaults
{<OntologyAttribute city.name>: (rdflib.term.Literal('John Smith'), False, None), <OntologyAttribute city.age>: (rdflib.term.Literal('25', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), False, None)}

You can get the non-inherited attributes and their defaults
{}
{<OntologyAttribute city.name>: (rdflib.term.Literal('John Smith'), False, None), <OntologyAttribute city.age>: (rdflib.term.Literal('25', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')), False, None)}


## Operations specific to ontology relationships

You can access the inverse, the domain and the range of relationships.

In [8]:
print("\nYou can get the inverse of a relationship")
print(city.hasInhabitant.inverse)


You can get the inverse of a relationship
city.INVERSE_OF_hasInhabitant


## Operations specific to attributes

You can acces the datatype and the argument name of attributes.

In [9]:
print("\nYou can get the argument name of an attribute. "
      "The argument name is used when instantiating CUDS objects")
print(city.age.argname)

print("\nYou can get the datatype of attributes")
print(city.age.datatype)

print("\nYou can use the attribute to convert values "
      "to the datatype of the attribute")
print(city.age.convert_to_datatype("10"))


You can get the argument name of an attribute. The argument name is used when instantiating CUDS objects
age

You can get the datatype of attributes
http://www.w3.org/2001/XMLSchema#integer

You can use the attribute to convert values to the datatype of the attribute
10


## CUDS

You can use ontology classes to create CUDS objects.

In [11]:
print("\nYou can instantiate CUDS objects using ontology classes")
print(city.Citizen(name="Test Person", age=42))
print("Take a look at api_example.py for a description of the CUDS API")

print("\nYou can check if a CUDS object is an instace of a ontology class")
print(city.Citizen(name="Test Person", age=42).is_a(city.Citizen))
print(city.Citizen(name="Test Person", age=42).is_a(city.LivingBeing))

print("\nYou can get the ontology class of a CUDS object.")
print(city.Citizen(name="Test Person", age=42).oclass)



You can instantiate CUDS objects using ontology classes
city.Citizen: 1714a778-5943-4611-af3a-e3b0cb32693d
Take a look at api_example.py for a description of the CUDS API

You can check if a CUDS object is an instace of a ontology class
True
True

You can get the ontology class of a CUDS object.
city.Citizen
